## loading

In [1]:
import pandas as pd

file_path = '/home/agatha/Desktop/MA3/sem proj/autoformalization_with_hypothesis/data/dependencies_mathlib_v4.14.0-rc1.jsonl'
df = pd.read_json(file_path, lines=True)

In [12]:
df.head()

,full_name,full_decl_no_comments,dependencies,file,in_mathlib
0,main,def main (args : List String) : IO UInt32 := d...,"[{'full_name': 'List', 'mod_name': 'Init.Prelu...",.lake/packages/lean4/src/lean/lake/LakeMain.lean,False
1,Lean.PrettyPrinter.Parenthesizer.mkAntiquot.pa...,@[export lean_mk_antiquot_parenthesizer]\ndef ...,"[{'full_name': 'String', 'mod_name': 'Init.Pre...",.lake/packages/lean4/src/lean/Lean/Parser.lean,False
2,Lean.PrettyPrinter.Parenthesizer.ident.parenth...,@[builtin_parenthesizer ident] def ident.paren...,[{'full_name': 'Lean.PrettyPrinter.Parenthesiz...,.lake/packages/lean4/src/lean/Lean/Parser.lean,False
3,Lean.PrettyPrinter.Parenthesizer.numLit.parent...,@[builtin_parenthesizer num] def numLit.parent...,[{'full_name': 'Lean.PrettyPrinter.Parenthesiz...,.lake/packages/lean4/src/lean/Lean/Parser.lean,False
4,Lean.PrettyPrinter.Parenthesizer.scientificLit...,@[builtin_parenthesizer scientific] def scient...,[{'full_name': 'Lean.PrettyPrinter.Parenthesiz...,.lake/packages/lean4/src/lean/Lean/Parser.lean,False


In [13]:
all_premises = df['full_decl_no_comments'].tolist()
print(len(all_premises))

200670


## encoding

In [4]:
import torch
import pickle
from typing import Union, List
from transformers import AutoTokenizer, AutoModelForTextEncoding

tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-retriever-byt5-small")
model = AutoModelForTextEncoding.from_pretrained("kaiyuy/leandojo-lean4-retriever-byt5-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
@torch.no_grad()
def original_encode(s: Union[str, List[str]]) -> torch.Tensor:
    """Encode texts into feature vectors."""
    if isinstance(s, str):
        s = [s]
        should_squeeze = True
    else:
        should_squeeze = False
    tokenized_s = tokenizer(s, return_tensors="pt", padding=True)
    hidden_state = model(tokenized_s.input_ids).last_hidden_state
    lens = tokenized_s.attention_mask.sum(dim=1)
    features = (hidden_state * tokenized_s.attention_mask.unsqueeze(2)).sum(dim=1) / lens.unsqueeze(1)
    if should_squeeze:
      features = features.squeeze()
    return features

In [6]:
@torch.no_grad()
def encode(s: List[str]) -> torch.Tensor:
    """Encode a list of texts into feature vectors."""
    tokenized_s = tokenizer(s, return_tensors="pt", padding=True, truncation=True)
    input_ids = tokenized_s.input_ids
    attention_mask = tokenized_s.attention_mask

    # Process inputs in batches to prevent memory overflow
    hidden_state = model(input_ids).last_hidden_state
    lens = attention_mask.sum(dim=1)
    features = (hidden_state * attention_mask.unsqueeze(2)).sum(dim=1) / lens.unsqueeze(1)
    return features

## batch encoding

In [7]:
import multiprocessing
from tqdm import tqdm

In [8]:
def encode_premises(premises: List[str], filename: str, batch_size: int = 8):
    """Encode all the premises in batches and save to a file."""
    premise_encs = []
    print(f"Encoding premises in batches of {batch_size}...")
    for i in tqdm(range(0, len(premises), batch_size)):
        batch = premises[i:i + batch_size]
        enc = encode(batch)
        premise_encs.append(enc.cpu())
    premise_encs = torch.cat(premise_encs, dim=0)
    premise_encs = premise_encs.numpy()
    data = {'premises': premises, 'encodings': premise_encs}
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Premises and encodings saved to {filename}")

In [9]:
model.eval()
# Set PyTorch to use all available CPU cores
num_cpu_cores = multiprocessing.cpu_count()
torch.set_num_threads(num_cpu_cores)
print(f"Using {num_cpu_cores} CPU cores for processing.")

Using 12 CPU cores for processing.


In [10]:
encode_premises(all_premises, 'premises.pkl', batch_size=8)

Encoding premises in batches of 8...


  0%|          | 1/25084 [01:40<701:18:39, 100.65s/it]


KeyboardInterrupt: 

## pickle test

In [1]:
raise Exception("stop here")

Exception: stop here

In [25]:
import pickle
with open('premises.pkl', 'wb') as f:
    pickle.dump(data, f)

In [26]:
with open('premises.pkl', 'rb') as f:
    data = pickle.load(f)

premises = data['premises']
encodings = torch.tensor(data['encodings'])

In [27]:
print(premises)
print(encodings.shape)

['<a>vsub_eq_zero_iff_eq</a> @[simp] lemma vsub_eq_zero_iff_eq {p1 p2 : P} : p1 -ᵥ p2 = (0 : G) ↔ p1 = p2', "<a>is_scalar_tower.coe_to_alg_hom'</a> @[simp] lemma coe_to_alg_hom' : (to_alg_hom R S A : S → A) = algebra_map S A", '<a>polynomial.X_sub_C_ne_zero</a> theorem X_sub_C_ne_zero (r : R) : X - C r ≠ 0', '<a>forall_true_iff</a> theorem forall_true_iff : (α → true) ↔ true', 'def <a>Nat.gcd</a> : Nat → Nat → Nat\n| 0        y := y\n| (succ x) y := have y % succ x < succ x, from mod_lt _ $ succ_pos _,\n                gcd (y % succ x) (succ x)', '@[simp] theorem <a>Nat.gcd_zero_left</a> (x : Nat) : gcd 0 x = x', '@[simp] theorem <a>Nat.gcd_succ</a> (x y : Nat) : gcd (succ x) y = gcd (y % succ x) (succ x)', '@[simp] theorem <a>Nat.mod_self</a> (n : Nat) : n % n = 0']
torch.Size([8, 1472])
